In [13]:
import pandas as pd
import sqlite3
import warnings
import pyodbc
import numpy as np

warnings.simplefilter('ignore')
print(pyodbc.drivers())

['ODBC Driver 17 for SQL Server']


<h3> connecties leggen/inlezen van sqlite bestanden en csv files inlezen

In [14]:
connSales = sqlite3.connect('data/raw/go_sales_train2.sqlite')
connTrain = sqlite3.connect('data/raw/go_staff_train.sqlite')
connCRM = sqlite3.connect('data/raw/go_crm_train.sqlite')

# Pad naar de CSV-bestanden
inventory_levels_path = 'data/raw/inventory_levels_train.csv'
product_forecast_path = 'data/raw/product_forecast_train.csv' 

# Lees de CSV-bestanden in
inventory_levels_df = pd.read_csv(inventory_levels_path, index_col=0)
product_forecast_df = pd.read_csv(product_forecast_path)

# Bekijk de eerste paar rijen van de DataFrames
print(inventory_levels_df.head())
print(product_forecast_df.head())

for conn in [connSales, connTrain, connCRM]:
    print("SQLite3 Connection: ",conn)

   INVENTORY_YEAR  INVENTORY_MONTH  PRODUCT_NUMBER  INVENTORY_COUNT
0            2023                4              48             1932
1            2023                4              49             1400
2            2023                4              50            21705
3            2023                4              51             9710
4            2023                4              52             5616
   PRODUCT_NUMBER  YEAR  MONTH  EXPECTED_VOLUME
0              45  2023     12               90
1              46  2023     12               84
2              47  2023     12              468
3              48  2023     12               91
4              49  2023     12               35
SQLite3 Connection:  <sqlite3.Connection object at 0x113706e30>
SQLite3 Connection:  <sqlite3.Connection object at 0x113706d40>
SQLite3 Connection:  <sqlite3.Connection object at 0x1137058a0>


In [15]:
# Functie om alle tabellen uit een SQLite-database in te lezen
def load_sqlite_tables_to_dfs(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    dfs = {}
    for table_name in tables:
        table_name = table_name[0]
        dfs[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    return dfs


# Lees alle tabellen van alle SQLite-databases in
all_dfs = {
    'go_sales_train2.sqlite': load_sqlite_tables_to_dfs(connSales),
    'go_staff_train.sqlite': load_sqlite_tables_to_dfs(connTrain),
    'go_crm_train.sqlite': load_sqlite_tables_to_dfs(connCRM),
    'inventory_levels_train.csv': {'inventory_levels': inventory_levels_df},
    'product_forecast_train.csv': {'product_forecast': product_forecast_df}
}


# Voorbeeld: Bekijk de eerste paar rijen van een specifieke tabel in een specifieke database
db_name = 'go_sales_train2.sqlite'
table_name = 'sales_branch'  # Vervang dit door de naam van een tabel in de database
if db_name in all_dfs and table_name in all_dfs[db_name]:
    print(all_dfs[db_name][table_name].head())
else:
    print(f"Tabel {table_name} niet gevonden in database {db_name}")




 # Sluit de verbindingen wanneer je klaar bent
for conn in [connSales, connTrain, connCRM]:
    print("SQLite3 Connection: ", conn)
    conn.close()


   SALES_BRANCH_CODE                       ADDRESS1       ADDRESS2       CITY  \
0                  6  75, rue du Faubourg St-Honoré           None      Paris   
1                  7                Piazza Duomo, 1           None     Milano   
2                  9            Singelgravenplein 4  4e verdieping  Amsterdam   
3                 13                 Schwabentor 35           None    Hamburg   
4                 14               Leopoldstraße 36           None    München   

          REGION POSTAL_ZONE  COUNTRY_CODE  
0           None     F-75008             1  
1           None     I-20121             6  
2  Noord-Holland     1233 BW             7  
3           None     D-22529             2  
4           None     D-81241             2  
SQLite3 Connection:  <sqlite3.Connection object at 0x113706e30>
SQLite3 Connection:  <sqlite3.Connection object at 0x113706d40>
SQLite3 Connection:  <sqlite3.Connection object at 0x1137058a0>


<h1>Verbinden met sql server

In [1]:

# Stel je verbindingsgegevens in
server = '127.0.0.1'        
port = '1433'               
database = 'GreatOutdoors_SDM'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Maak de connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"  # Voorkomt SSL-warnings bij self-signed
    "Timeout=30;"
)

'''
try:
    # Verbind met de database
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    # Test: maak een query (bijv. checken of de verbinding werkt)
    cursor.execute("SELECT @@VERSION;")
    row = cursor.fetchone()
    print("SQL Server versie:", row[0])

    # Sluit de verbindingen wanneer je klaar bent
    cursor.close()
    conn.close()
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(f"OperationalError: {sqlstate}")
    
    '''

'\ntry:\n    # Verbind met de database\n    conn = pyodbc.connect(connection_string)\n    cursor = conn.cursor()\n\n    # Test: maak een query (bijv. checken of de verbinding werkt)\n    cursor.execute("SELECT @@VERSION;")\n    row = cursor.fetchone()\n    print("SQL Server versie:", row[0])\n\n    # Sluit de verbindingen wanneer je klaar bent\n    cursor.close()\n    conn.close()\nexcept pyodbc.Error as ex:\n    sqlstate = ex.args[1]\n    print(f"OperationalError: {sqlstate}")\n    \n    '

<h4> dict testen


In [17]:
# Print de eerste paar rijen van elke DataFrame in de dictionary
for db_name, tables in all_dfs.items():
    print(f"\nDatabase/CSV: {db_name}")
    for table_name, df in tables.items():
        print(f"\nTable: {table_name}")
        print(df.head())



# merge sales branch in go_sales_train2.sqlite en go_staff_train.sqlite

#merge sales staff in go_sales_train2.sqlite en go_staff_train.sqlite

#de twee “country”-definities zijn samengevoegd tot één tabel "Country" met zowel de verkoop- als CRM-attributen.
#De tabel “crm_country” en “crm_retailer_site” zijn geheel verwijderd.


Database/CSV: go_sales_train2.sqlite

Table: country
   COUNTRY_CODE        COUNTRY LANGUAGE CURRENCY_NAME
0             1         France       EN        francs
1             2        Germany       EN         marks
2             3  United States       EN       dollars
3             4         Canada       EN       dollars
4             5        Austria       EN    schillings

Table: order_details
   ORDER_DETAIL_CODE  ORDER_NUMBER  PRODUCT_NUMBER  QUANTITY  UNIT_COST  \
0              47737          1638               1        84       4.38   
1              47738          1159               1       126       4.38   
2              47739          1171               1       122       4.38   
3              47740          1154               1       122       4.38   
4              47741          1205               1       272       4.38   

   UNIT_PRICE  UNIT_SALE_PRICE  
0        6.59             6.59  
1        6.59             6.59  
2        6.59             6.03  
3        6.59    

<h2> merge sales_staff en sales_branch
<h3> dit was volledig overbodig aangezien de tabellen uniek waren, 

In [18]:
# Haal de sales_branch tabellen uit de twee databases
sales_branch_sales = all_dfs['go_sales_train2.sqlite']['sales_branch']
sales_branch_staff = all_dfs['go_staff_train.sqlite']['sales_branch']

# Zet de twee DataFrames onder elkaar (concatenatie)
merged_sales_branch = pd.concat([sales_branch_sales, sales_branch_staff], ignore_index=True)
# Verwijder eventueel duplicaten op basis van de primaire sleutel (bijv. SALES_BRANCH_CODE)
merged_sales_branch.drop_duplicates(subset='SALES_BRANCH_CODE', inplace=True)

# Voor sales_staff:

sales_staff_staff = all_dfs['go_staff_train.sqlite']['sales_staff']

print (sales_staff_staff)


     SALES_STAFF_CODE FIRST_NAME  LAST_NAME                   POSITION_EN  \
0                   4      Denis       Pagé                Branch Manager   
1                   5  Élizabeth     Michel  Level 3 Sales Representative   
2                   6      Émile   Clermont  Level 1 Sales Representative   
3                   7    Étienne     Jauvin  Level 2 Sales Representative   
4                  12    Elsbeth  Wiesinger  Level 2 Sales Representative   
..                ...        ...        ...                           ...   
97                120      Giele   Laermans  Level 1 Sales Representative   
98                121   François    De Crée  Level 1 Sales Representative   
99                122     Yvette    Lattrez  Level 3 Sales Representative   
100               123      Willi  Seefelder  Level 2 Sales Representative   
101               124     Sabine     Grüner  Level 3 Sales Representative   

            WORK_PHONE  EXTENSION                FAX                   EMAI

<h2> merge country

In [19]:
# Laad beide country DataFrames
crm_country = all_dfs['go_crm_train.sqlite']['country']
sales_country = all_dfs['go_sales_train2.sqlite']['country']

# Hernoem de kolom in sales_country
sales_country = sales_country.rename(columns={'COUNTRY': 'COUNTRY_EN_SALES'})

# Hernoem de kolom in crm_country (als deze ook COUNTRY_EN heet)
crm_country = crm_country.rename(columns={'COUNTRY_EN': 'COUNTRY_EN_CRM'})

# Merge op COUNTRY_CODE; we combineren de velden zodat we alle attributen hebben
merged_country = pd.merge(sales_country, crm_country, on='COUNTRY_CODE', how='outer')

#merge country_en en country
merged_country['COUNTRY_EN'] = (
    merged_country['COUNTRY_EN_SALES']
    .fillna(merged_country['COUNTRY_EN_CRM'])
)
#verwijder overbodige kolommen
merged_country = merged_country.drop(
    columns=['COUNTRY_EN_SALES', 'COUNTRY_EN_CRM']
)

# Definieer de gewenste kolomvolgorde
gewenste_volgorde = [
    'COUNTRY_CODE',  
    'COUNTRY_EN',    
    'LANGUAGE',      
    'CURRENCY_NAME',
    'FLAG_IMAGE',   
    'SALES_TERRITORY_CODE',
]

# Pas de volgorde toe op de DataFrame
merged_country = merged_country[gewenste_volgorde] 


# Print de aangepaste DataFrame
print(merged_country)





    COUNTRY_CODE      COUNTRY_EN LANGUAGE CURRENCY_NAME FLAG_IMAGE  \
0              1          France       EN        francs        F01   
1              2         Germany       EN         marks        F02   
2              3   United States       EN       dollars        F03   
3              4          Canada       EN       dollars        F04   
4              5         Austria       EN    schillings        F05   
5              6           Italy       EN          lira        F06   
6              7     Netherlands       EN      guilders        F07   
7              8     Switzerland       EN        francs        F08   
8              9  United Kingdom       EN        pounds        F09   
9             10          Sweden       EN         krona        F10   
10            11           Japan       EN           yen        F11   
11            12          Taiwan       EN    new dollar        F12   
12            13           Korea       EN           won        F13   
13            14    

<h3> verwijder country en country uit dictionary, en vervang met merged country

<h4> ook met sales branch en sales_staff


In [20]:
#verwijder sales_staff uit go sales
if 'sales_staff' in all_dfs['go_sales_train2.sqlite']:
    del all_dfs['go_sales_train2.sqlite']['sales_staff']
    print("gelukt!")

# Verwijder sales_branch uit go_sales
if 'sales_branch' in all_dfs['go_sales_train2.sqlite']:
    del all_dfs['go_sales_train2.sqlite']['sales_branch']
    print("gelukt!")

gelukt!
gelukt!


In [21]:
#vervang country in go sales met merged country
all_dfs['go_sales_train2.sqlite']['country'] = merged_country

# Verwijder country uit go_crm_train.sqlite (als je deze niet meer apart wilt behouden)
if 'country' in all_dfs['go_crm_train.sqlite']:
    del all_dfs['go_crm_train.sqlite']['country']
    print("gelukt!")

#verwijder retailer_site uit go_crm_train.sqlite
if 'retailer_site' in all_dfs['go_crm_train.sqlite']:
    del all_dfs['go_crm_train.sqlite']['retailer_site']
    print("gelukt!")

gelukt!
gelukt!


<h2> check of alle tabellen aanwezig zijn

In [22]:
# Loop door elke database in all_dfs
for db_name, tables in all_dfs.items():
    print(f"\nDatabase/CSV: {db_name}")
    # Loop door elke tabel in de database
    for table_name, df in tables.items():
        # Bereken het aantal rijen en kolommen in de tabel
        row_count = len(df)
        col_count = len(df.columns)
        kolomnamen = df.columns.tolist()  # Lijst van kolomnamen

        # Print de informatie
        print(f"Tabel: {table_name}")
        print(f"  Aantal rijen: {row_count}")
        print(f"  Aantal kolommen: {col_count}")
        print(f"  Kolomnamen: {kolomnamen}")


Database/CSV: go_sales_train2.sqlite
Tabel: country
  Aantal rijen: 21
  Aantal kolommen: 6
  Kolomnamen: ['COUNTRY_CODE', 'COUNTRY_EN', 'LANGUAGE', 'CURRENCY_NAME', 'FLAG_IMAGE', 'SALES_TERRITORY_CODE']
Tabel: order_details
  Aantal rijen: 37757
  Aantal kolommen: 7
  Kolomnamen: ['ORDER_DETAIL_CODE', 'ORDER_NUMBER', 'PRODUCT_NUMBER', 'QUANTITY', 'UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE']
Tabel: order_header
  Aantal rijen: 4784
  Aantal kolommen: 8
  Kolomnamen: ['ORDER_NUMBER', 'RETAILER_NAME', 'RETAILER_SITE_CODE', 'RETAILER_CONTACT_CODE', 'SALES_STAFF_CODE', 'SALES_BRANCH_CODE', 'ORDER_DATE', 'ORDER_METHOD_CODE']
Tabel: order_method
  Aantal rijen: 7
  Aantal kolommen: 2
  Kolomnamen: ['ORDER_METHOD_CODE', 'ORDER_METHOD_EN']
Tabel: product
  Aantal rijen: 115
  Aantal kolommen: 9
  Kolomnamen: ['PRODUCT_NUMBER', 'INTRODUCTION_DATE', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN', 'PRODUCT_IMAGE', 'LANGUAGE', 'PRODUCT_NAME', 'DESCRIPTION']
Tabel: product_line
  Aantal rijen: 

<h2> sql server db vullen

In [23]:
import pandas as pd
import pyodbc

def clean_nan_values(all_dfs):
    for db_name, tables in all_dfs.items():
        for table_name, df in tables.items():
            df = df.astype(object).where(pd.notnull(df), None)  # Fix NULLs
            all_dfs[db_name][table_name] = df
    return all_dfs

# Pas toe op je dictionary
all_dfs = clean_nan_values(all_dfs)


def upload_dataframes_to_sql(all_dfs):
    try:
        # Maak verbinding met de database
        with pyodbc.connect(connection_string, autocommit=False) as conn:
            cursor = conn.cursor()

            print("⏳ Uitschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'")
            conn.commit()

            # Loop door elke database/csv in all_dfs
            for db_name, tables in all_dfs.items():
                print(f"\nBezig met uploaden van {db_name}...")
                
                # Loop door elke tabel in de database/csv
                for table_name, df in tables.items():
                    print(f"\nTabel: {table_name}")
                    
                    # Genereer de kolomnamen voor de INSERT-query
                    columns = ', '.join([f'[{col}]' for col in df.columns])
                    placeholders = ', '.join(['?'] * len(df.columns))
                    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    
                    # Loop door elke rij in de DataFrame
                    for index, row in df.iterrows():
                        try:
                            row_data = tuple(row)
                            cursor.execute(insert_query, row_data)
                            conn.commit()  # Commit na elke rij
                            print(f"  Rij {index + 1} ✅ Upload voltooid")
                        except pyodbc.Error as e:
                            conn.rollback()  # Rollback bij fouten
                            error_msg = str(e).replace('\n', ' ')
                            print(f"  Rij {index + 1} ❌ Fout: {error_msg}")
                            continue
            
            print("\n⏳ Herinschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'")
            conn.commit()

            print("\n🎉 Upload voltooid voor alle tabellen!")
            cursor.close()
            conn.close()
            
    except pyodbc.Error as e:
        print(f"❌ Databasefout: {str(e)}")

# Roep de functie aan met je all_dfs dictionary
upload_dataframes_to_sql(all_dfs)


⏳ Uitschakelen van FOREIGN KEY constraints...

Bezig met uploaden van go_sales_train2.sqlite...

Tabel: country
  Rij 1 ❌ Fout: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__country__FC7F59079386D156'. Cannot insert duplicate key in object 'dbo.country'. The duplicate key value is (1). (2627) (SQLExecDirectW)")
  Rij 2 ❌ Fout: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__country__FC7F59079386D156'. Cannot insert duplicate key in object 'dbo.country'. The duplicate key value is (2). (2627) (SQLExecDirectW)")
  Rij 3 ❌ Fout: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__country__FC7F59079386D156'. Cannot insert duplicate key in object 'dbo.country'. The duplicate key value is (3). (2627) (SQLExecDirectW)")
  Rij 4 ❌ Fout: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL